In [ ]:
# pip install instructor

In [ ]:
import requests
import time
import re
from typing import List
import json

### Original Version:

In [ ]:
def LLaMA_response(messages, model_name, url="http://localhost:11434/api/generate"):
    """
    LLM module to be called

    Args
        messages: list of message dictionaries following ChatCompletion format
        model_name: name of the LLaMA model
        url: endpoint where LLaMA is hosted
    """
    prompt = "\n".join(
        [f"{msg['role'].capitalize()}: {msg['content']}" for msg in messages]
    )
    data = {
        "model": model_name,
        "prompt": prompt,
        # "max_tokens": 2000,
        "temperature": 0.0,
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "stream": False,
    }

    try:
        response = requests.post(url=url, json=data)

        if response.status_code == 200:
            response_text = response.json().get("response", "")
    
            # print(f"Token_num_count: {token_num_count}")
            return response_text
        else:
            print("Error:", response.status_code, response.json())
            return None
    except Exception as e:
        print(f"API call failed: {e}")
        return None

In [ ]:
prompt = """
        You are a central planner directing agent in a grid-like field to move colored boxes.
        You are agent at grid [0.5, 1.5].
        
        The goals and rules of this environment are:
        You are an agent in a grid-like field to move colored boxes.
        Each agent is assigned to a 1x1 square and can only interact with objects in its area.
        Agents can move a box to a neighboring square or a same-color target.
        You can only move same color boxes to same color targets.
        Each square can contain many targets and boxes.
        The squares are identified by their center coordinates, e.g., square[0.5, 0.5].
        Actions are like: move(box_red, target_red) or move(box_red, square[0.5, 0.5]).
        When planning for action, remanber to not purely repeat the actions but learn why the state changes or remains in a dead loop.
        Avoid being stuck in action loops.
        Additionally, when there is a box still in the grid, the agent in this grid have to make an action in the next step.
        when there is a box still in the grid, the agent in this grid have to make an action in the next step.
        
        Your task is to instruct each agent to match all boxes to their color-coded targets.
        After each move, agents provide updates for the next sequence of actions.
        
        You are the central agent and your job is to coordinate the agents optimally.
        The previous state and action pairs at each step are: 
        State1: {'0.5, 0.5': ['target_blue', 'target_orange', 'box_orange', 'target_orange', 'target_orange'], 
        '0.5, 1.5': ['box_blue', 'target_blue', 'box_red', 'target_purple'], 
        '1.5, 0.5': ['box_red', 'target_red', 'box_green', 'target_green', 'box_orange'], 
        '1.5, 1.5': ['box_blue', 'target_red', 'box_purple', 'box_orange']
        }
        Action1: {"Agent[0.5, 0.5]": "move(box_orange, target_orange)", "Agent[0.5, 1.5]": "move(box_blue, square[0.5, 0.5])", "Agent[1.5, 0.5]": "move(box_red, square[0.5, 0.5])"}

        Hence, the current state is {'0.5_0.5': ['target_blue', 'target_orange', 'target_orange', 'box_blue', 'box_red'], '0.5_1.5': ['target_blue', 'box_red', 'target_purple'], '1.5_0.5': ['target_red', 'box_green', 'target_green', 'box_orange'], '1.5_1.5': ['box_blue', 'target_red', 'box_purple', 'box_orange']}, with the possible actions: 
        Agent[0.5, 0.5]: I am in square[0.5, 0.5], I can observe ['target_blue', 'target_orange', 'target_orange', 'box_blue', 'box_red'], I can do ['move(box_blue, square[1.5, 0.5])', 'move(box_blue, square[0.5, 1.5])', 'move(box_blue, target_blue)', 'move(box_red, square[1.5, 0.5])', 'move(box_red, square[0.5, 1.5])']
        Agent[0.5, 1.5]: I am in square[0.5, 1.5], I can observe ['target_blue', 'box_red', 'target_purple'], I can do ['move(box_red, square[1.5, 1.5])', 'move(box_red, square[0.5, 0.5])']
        Agent[1.5, 0.5]: I am in square[1.5, 0.5], I can observe ['target_red', 'box_green', 'target_green', 'box_orange'], I can do ['move(box_green, square[0.5, 0.5])', 'move(box_green, square[1.5, 1.5])', 'move(box_green, target_green)', 'move(box_orange, square[0.5, 0.5])', 'move(box_orange, square[1.5, 1.5])']
        Agent[1.5, 1.5]: I am in square[1.5, 1.5], I can observe ['box_blue', 'target_red', 'box_purple', 'box_orange'], I can do ['move(box_blue, square[0.5, 1.5])', 'move(box_blue, square[1.5, 0.5])', 'move(box_purple, square[0.5, 1.5])', 'move(box_purple, square[1.5, 0.5])', 'move(box_orange, square[0.5, 1.5])', 'move(box_orange, square[1.5, 0.5])'].
        
        Think about what the future 5 actions would be if you want to achieve the goal.
        Remanber to wirte out for each step, what you plan for every agent to do and what would the consequences state change be.

        Based on this, generate the action plan for the immediate next step for each agent.

        Specify your action plan in this format: {"Agent[0.5, 0.5]":"move(box_blue, square[0.5, 1.5])","Agent[0.5, 1.5]": "move(box_blue, target_blue)"}.
        
        Include an agent only if it has a task next. 

        """

In [ ]:
model_name = 'qwen2.5:7b-instruct-q5_K_M'
messages =  [{'role':'system', 
              'content': '''You are a helpful assistant.

              Example Action Plan:
              {"Agent[0.5, 0.5]":"move(box_blue, square[0.5, 1.5])", "Agent[1.5, 0.5]":"move(box_blue, target_blue)}
                 
                Make sure that:
                - If no action for an agent in the next step, do not include it in JSON output. 
                - At most one action for each agent in each step.
                '''
             }, 
             {"role": "user",
              "content": prompt,
             }
            ]
#response = LLaMA_response(messages, model_name)
#response

### Instructor Version

In [ ]:
from typing import Dict
from pydantic import BaseModel, Field
import instructor

class ActionPlan(BaseModel):
    justification: str
    future_step: str
    actions_plan: Dict

    # @field_validator('actions')
    # def validate_actions(cls, actions: Dict[str, str]) -> Dict[str, str]:
    #     # Validate agent position format
        
    #     agent_pattern = r'Agent\[\d+\.\d+, \d+\.\d+]'
    #     # Validate action format
    #     # action_pattern = r'^move\((.*?), \s(.*?)\)'
        
    #     formatted_actions = {}
        
    #     # Convert keys to proper format if they're using underscores
    #     for key, value in v.items():
    #         # If key is in format "0.5_0.5", convert to "Agent[0.5, 0.5]"
    #         if '_' in key:
    #             x, y = key.split('_')
    #             new_key = f"Agent[{x}, {y}]"
    #         else:
    #             new_key = key
                
    #         # Validate the format
    #         if not re.match(agent_pattern, new_key):
    #             raise ValueError(f"Invalid agent position format: {new_key}")
            
    #         formatted_actions[new_key] = value

    #     return actions

In [ ]:
print((ActionPlan.schema_json(indent=2)))

In [ ]:
class ActionPlan(BaseModel):
    justification: str
    future_step: str
    actions_plan: Dict

def LLaMA_response_instructor(messages, model_name, response_model, url="http://localhost:11434/v1"):
    """
    LLM module to be called

    Args
        messages: list of message dictionaries following ChatCompletion format
        model_name: name of the LLaMA model
        url: endpoint where LLaMA is hosted
    """
    MAX_RETRY = 7
    count = 0
    try:
        # enables `response_model` in create call
        client = instructor.from_openai(
            OpenAI(
                base_url=url,
                api_key="ollama",  # required, but unused
            ),
            mode=instructor.Mode.JSON,
        )
        while True:
            if count >= MAX_RETRY:
                print(f'''Max retries reach. LLM failed at outputing the correct result. 
                Input parameter response_model might have issue: {response_model.schema_json(indent=2)}''')
                raise ValidationError
            try:
                # Use instructor to handle the structured response
                response = client.chat.completions.create(
                    model=model_name,
                    messages=messages,
                    response_model=response_model,  # Use the Character model to structure the response
                )
        
                # Print the structured output as JSON
                response = response.model_dump_json(indent=2)
                print(response)  # This outputs the structured response
                return response
            except ValidationError as ve:
                count += 1
                print(f"Validation failed: {ve} for {count} times. Retrying...")

    except Exception as e:
        print(f"API call failed: {e}")
        return None


In [ ]:
model_name = 'qwen2.5:7b-instruct-q5_K_M'
messages =  [{'role':'system', 
              'content': '''You are a helpful assistant.

              Example Action Plan:
              {"Agent[0.5, 0.5]":"move(box_blue, square[0.5, 1.5])", "Agent[1.5, 0.5]":"move(box_blue, target_blue)}
                 
                Make sure that:
                - If no action for an agent in the next step, do not include it in JSON output. 
                - At most one action for each agent in each step.
                '''
             }, 
             {"role": "user",
              "content": prompt,
             }
            ]

response = LLaMA_response_instructor(messages, model_name, ActionPlan, url="http://localhost:11434/v1")

In [ ]:
#response_json = json.loads(response)
response['actions_plan']

In [ ]:
# def process_response(response: str) -> dict:
#     """
#     Processes a raw response string containing agent locations and actions,
#     extracts relevant information, and converts it into dictionary format for loading in json.

#     Args:
#         raw_response (str): The string is expected to include an "EXECUTE" keyword followed by
#             JSON-like content describing agent locations and actions
#             (e.g "EXECUTE
#             {"Agent[0.5, 0.5]":"move(box_blue, square[0.5, 1.5])")

#     Returns:
#         dict: A dictionary where keys are agent identifiers (e.g., "Agent[0.5, 0.5]")
#               and values are actions (e.g., "move(box_blue, square[0.5, 1.5])").
#     Note:
#         Return input response back if error in parsing the response.
#     """

#     try:
#         pattern_1 = r"agent\[(\d+\.\d+), (\d+\.\d+)\]"
#         agent_loc = re.findall(pattern_1, response, re.IGNORECASE)

#         pattern_2 = r"move\((.*?),\s(.*?)\)"  # r'move(\(\w+, (?:square.\d+\.\d+, \d+\.\d+.|\w+)\))'
#         agent_action = re.findall(pattern_2, response, re.IGNORECASE)

#     except:
#         return response
#         # raise ValueError(f'Error in parsing the response: {response}')

#     num_agent = len(agent_loc)
#     action = []
#     if len(agent_loc) == len(agent_action):
#         for i in range(num_agent):
#             action.append(
#                 f'"Agent[{agent_loc[i][0]}, {agent_loc[i][1]}]": "move({agent_action[i][0]}, {agent_action[i][1]})"'
#             )

#         json_str = "{" + ", ".join(action) + "}"
#     else:
#         print(f"Agent-action pair is inconsistent: {response}")
#         return response
#         # raise ValueError(f'Agent-action pair is inconsistent: {response}')

#     return json_str


In [ ]:
print(
    "\n".join([f"{k}: {v}" for k, v, in json.loads(response).items()])
)

In [ ]:
json.loads(response)['actions_plan']

In [ ]:
a = '''Agent[0.5, 0.5]: I am in square[0.5, 0.5], I can observe ['target_blue', 'target_orange', 'target_orange', 'box_blue', 'box_red'], I can do ['move(box_blue, square[1.5, 0.5])', 'move(box_blue, square[0.5, 1.5])', 'move(box_blue, target_blue)', 'move(box_red, square[1.5, 0.5])', 'move(box_red, square[0.5, 1.5])']
Agent[0.5, 1.5]: I am in square[0.5, 1.5], I can observe ['target_blue', 'box_red', 'target_purple'], I can do ['move(box_red, square[1.5, 1.5])', 'move(box_red, square[0.5, 0.5])']
Agent[1.5, 0.5]: I am in square[1.5, 0.5], I can observe ['target_red', 'box_green', 'target_green', 'box_orange'], I can do ['move(box_green, square[0.5, 0.5])', 'move(box_green, square[1.5, 1.5])', 'move(box_green, target_green)', 'move(box_orange, square[0.5, 0.5])', 'move(box_orange, square[1.5, 1.5])']
Agent[1.5, 1.5]: I am in square[1.5, 1.5], I can observe ['box_blue', 'target_red', 'box_purple', 'box_orange'], I can do ['move(box_blue, square[0.5, 1.5])', 'move(box_blue, square[1.5, 0.5])', 'move(box_purple, square[0.5, 1.5])', 'move(box_purple, square[1.5, 0.5])', 'move(box_orange, square[0.5, 1.5])', 'move(box_orange, square[1.5, 0.5])'].'''

dic = dict()
for b in a.split('Agent')[1:]:
    dic['Agent'+b[:10]] = eval(b[b.find('I can do')+9:].strip('\n.'))

def create_dynamic_pydantic_model(data: Dict[str, List[str]]) -> BaseModel:
"""
    Create a dynamic Pydantic model to constrain the LLM's output.
    The output must include one of the dictionary keys and one of its corresponding values.
    """
    # Generate a Literal type for the keys
    keys = Literal[tuple(data.keys())]
    values = Literal[tuple([v for v_list in dic.values() for v in v_list])]

    # Create a dynamic Pydantic model
    class DynamicModel(BaseModel):
        key: keys = Field(..., description="A key from the input dictionary.")
        value: str = Field(..., description="A value corresponding to the chosen key.")

    return DynamicModel

agent_to_action = create_dynamic_pydantic_model(dic)